In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

from time import sleep, time
from threading import Thread
from scipy.cluster.hierarchy import fcluster
from scipy.stats import zscore

from data_processing import DataIterator
from testing import TestFactory, ClusteredInfo
from dtw import dtw as cur_dtw
from dtw_wrapper import DtwWrapper

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
def norm_1(x, y):
    return np.linalg.norm(x - y, ord=1)
def norm_2(x, y):
    return np.linalg.norm(x - y, ord=2)

# Rewriting data in our format

In [3]:
# data = pd.read_csv("../data/preprocessed_large.csv", header=None)

# indexes = pd.Index(range(0, 200))
# data_preprocessed = pd.DataFrame(columns=["obj", "ch", "label", *indexes])

# l = []
# for j in data.index:
#     df = pd.DataFrame({"obj": [j for i in range(3)], 
#                   "label": [data.loc[j, 0] for i in range(3)]},
#                  columns=["obj", "label", *indexes])
#     df.index.name = "ch"
#     df.loc[0, indexes] = data.loc[j, 1:200].values
#     df.loc[1, indexes] = data.loc[j, 201:400].values
#     df.loc[2, indexes] = data.loc[j, 401:600].values 
#     df.reset_index(inplace=True)
#     data_preprocessed = data_preprocessed.append(df, sort=False)
    
# data_preprocessed.reset_index(drop=True, inplace=True)


In [7]:
data = pd.read_csv("../data/preprocessed_3ch_50point_overlap0.csv", index_col=0)
it = DataIterator(data, 50, random_state=42)
tests = TestFactory(it=it)
x = tests.set_sample(2048)

In [8]:
wrap = DtwWrapper(tests.X, hash(tests.infos), cur_dtw, norm_2, dtw_args={"z_normalize": True, "l": 0.2})
wrap.fill_distances(n_threads=8)

Loaded
dump
0
dump
30
dump
60
dump
90
dump
120
dump
150
dump
180
dump
210
dump
240
dump
270
dump
300
dump
330
dump
360
dump
390
dump
420
dump
450
dump
480
dump
510
dump
540
dump
570
dump
600
dump
630
dump
660
dump
690
dump
720
dump
750
dump
780
dump
810
dump
840
dump
870
dump
900
dump
930
dump
960
dump
990
41046.39708209038


In [9]:
tmp = np.genfromtxt("../data/distances/dtwnorm_2z_normalizeTruel0.2-43544539162478229291.csv")
(tmp < 0).sum() / (tmp != -1000).sum()

0.0

In [15]:
DtwWrapper(tests.X, hash(tests.infos), cur_dtw, norm_2, dtw_args={"z_normalize": True, "l": 0.2})
info = tests.test_dtw(cur_dtw, norm_2, dtw_args={"z_normalize": True, "l": 0.2})
info.cluster(60)
info.stats
index = info.stats.head(10).index.values
classifier_stat = {}
for i in info.stats.index:
    classifier_stat[i] = pd.Series(info.label[np.where(info.clusters_labels == i)[0]]).value_counts()
    
pd.DataFrame(classifier_stat).fillna(0).iloc[:, :20]

Loaded
Loaded
Elapsed time: 4.555


,23,15,49,20,51,33,43,35,32,24,11,48,31,25,40,47,56,26,29,21
1,4,72.0,0.0,0.0,3,23,19,6.0,41.0,0.0,24.0,52.0,2.0,33.0,14.0,2.0,0.0,7.0,0.0,0.0
2,71,0.0,68.0,35.0,46,23,8,33.0,0.0,14.0,1.0,0.0,6.0,0.0,0.0,0.0,8.0,0.0,4.0,4.0
3,24,9.0,10.0,9.0,5,11,13,6.0,14.0,3.0,9.0,2.0,8.0,3.0,8.0,1.0,11.0,1.0,27.0,8.0
4,37,5.0,11.0,34.0,7,8,13,18.0,3.0,30.0,0.0,0.0,36.0,0.0,2.0,6.0,4.0,24.0,7.0,20.0
5,2,2.0,1.0,0.0,2,1,7,0.0,0.0,5.0,11.0,0.0,1.0,9.0,8.0,14.0,9.0,7.0,2.0,4.0
6,2,4.0,0.0,1.0,8,3,6,0.0,2.0,6.0,9.0,0.0,0.0,5.0,13.0,22.0,12.0,4.0,0.0,0.0


In [4]:
tmp.shape

(2048, 2048)

In [6]:
1 % 4 == 1

True